# Project 3 - BD Analytics

In [1]:
from pyspark.context import SparkContext # for RDDs
from pyspark.sql import SparkSession # for DFs

spark = (SparkSession.builder
                    .appName('BDM_project3')
                    .getOrCreate()
        ) # for DFs

## Importing the data

In [2]:
papers_df = (spark.read
             .option("inferSchema", True) # Letting Spark itself define the schema
             .json("dblp-ref-0.json")
            )

papers_df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- year: long (nullable = true)



## Exploratory data analysis

In [3]:
# Subset of the dataframe
papers_df.show(5)

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|            abstract|             authors|                  id|n_citation|          references|               title|               venue|year|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----+
|The purpose of th...|[Makoto Satoh, Ry...|00127ee2-cb05-48c...|         0|[51c7e02e-f5ed-43...|Preliminary Desig...|international con...|2013|
|This paper descri...|[Gareth Beale, Gr...|001c58d3-26ad-46b...|        50|[10482dd3-4642-41...|A methodology for...|visual analytics ...|2011|
|This article appl...|[Altaf Hossain, F...|001c8744-73c4-4b0...|        50|[2d84c0f2-e656-4c...|Comparison of GAR...|pattern recogniti...|2009|
|                NULL|[Jea-Bum Park, By...|00338203-9eb3-40c...|         0|[8c78e4b0-632b-42...|Development of Re...|                   

In [4]:
# First row
papers_df.show(n=1, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 abstract   | The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication net- works. More specifically, we focus on the basic principles of network proto- cols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols. 
 authors    | [Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Kazunori Itoh, Masami Hashimoto, Makoto Otani, Michio Shimizu, Masahiko Sugimoto]                                                                       

In [5]:
# Summary statistics (for numeric columns)
papers_df.describe("n_citation", "year").toPandas()

,summary,n_citation,year
0,count,1000000,1000000
1,mean,30.87573,2006.365124
2,stddev,137.94247310778852,7.83383289543257
3,min,0,1937
4,max,73362,2017


In [11]:
# Extracting features

# Add more??

## Data Preprocessing

In [45]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=1e223a6e9370be68f8ca4df1e53e004bc6843ed2057d7050e416896c433bbdf7
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [46]:
# Only keeping the English documents

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from langdetect import detect, LangDetectException
# Only keeping the English documents
def detect_language(text):
    if text:
        try:
            return detect(text)
        except:
            return 'unknown'
    return 'unknown'
detect_language_udf = udf(detect_language, StringType())
papers_df = papers_df.withColumn("language", detect_language_udf(papers_df.abstract))
english_papers_df = papers_df.filter(papers_df.language == 'en')
#english_papers_df.select("title", "language").show(truncate=False)

In [15]:
# Removing stopwords (with Gensim)
from gensim.parsing.preprocessing import remove_stopwords #!pip install gensim

def remove_stop_words(text):
    return remove_stopwords(text)

In [18]:
# Remove custom stopwords
custom_stop_words = ['doi', 'purpose']
def remove_custom_stop_words(text):
    words = text.split() 
    filtered_words = [word for word in words if word not in custom_stop_words]  
    return ' '.join(filtered_words)


In [9]:
# Remove punctuation
# Maybe do not use - another implementation down below
import re

def remove_punctuation(text):
    p = re.compile('!()-[]{};:\'"\,<>./?@#$%^&*_~')
    m = p.match(text)
    return m

In [42]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import lower
from pyspark.sql.functions import regexp_extract, col

# Create a user-defined function (UDF)
remove_stop_words_udf = F.udf(remove_stop_words, StringType()) # Default return type is string
stop_words_udf = F.udf(remove_custom_stop_words, StringType()) # Default return type is string
#remove_punctuation_udf = F.udf(remove_punctuation, StringType()) # Default return type is string

# Apply the UDF 

# Convert into a lowercase
papers_df = papers_df.withColumn('abstract', lower(papers_df['abstract']))
papers_df = papers_df.withColumn('title', lower(papers_df['title']))

# Remove stop words
papers_df = papers_df.withColumn("abstract", remove_stop_words_udf(papers_df["abstract"]))
papers_df = papers_df.withColumn("title", remove_stop_words_udf(papers_df["title"]))

# Remove custom stop words
#papers_df = papers_df.withColumn("abstract", remove_custom_stop_words(papers_df["abstract"]))
#papers_df = papers_df.withColumn("title", remove_custom_stop_words(papers_df["title"]))

# Remove punctuation - why it doesn't work?
papers_df = papers_df.withColumn('abstract', regexp_extract(col('abstract'), '!()-[]{};:"\,<>./?@#$%^&*_~', 1))
papers_df = papers_df.withColumn('title', regexp_extract(col('title'), '!()-[]{};:"\,<>./?@#$%^&*_~', 1))

In [43]:
papers_df.show(n=1, truncate=False, vertical=True)

Py4JJavaError: An error occurred while calling o461.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 33) (c426a59c2c9b executor driver): org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.PATTERN] The value of parameter(s) `regexp` in `regexp_extract` is invalid: '!()-[]{};:\'"\\,<>./?@#$%^&*_~'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidPatternError(QueryExecutionErrors.scala:2619)
	at org.apache.spark.sql.errors.QueryExecutionErrors.invalidPatternError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.RegExpExtract_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$10(EvalPythonExec.scala:127)
	at scala.collection.It

## Vectorization

In [ ]:
# Converting data for ML algorithms

## Clustering

In [ ]:
# K-means and elbow method

## Search engine

In [ ]:
# Recommender function